In [41]:
import numpy as np
import pandas as pd
import os
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, SubsetRandomSampler

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cpu device


In [43]:
df = pd.read_excel("data.xlsx")

In [53]:
class NeuralNetwork(nn.Module):
  def __init__(self):
      super().__init__()
      self.linear_relu_stack = nn.Sequential(
          nn.Linear(4, 32),
          nn.ReLU(),
          nn.Linear(32, 16),
          nn.ReLU(),
          nn.Linear(16, 8),
          nn.ReLU(),
          nn.Linear(8, 4),
          nn.ReLU(),
          nn.Linear(4, 1),
      )

  def forward(self, x_tensor):
      logits = self.linear_relu_stack(x_tensor)
      return logits


In [45]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=8, bias=True)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=4, bias=True)
    (7): ReLU()
    (8): Linear(in_features=4, out_features=1, bias=True)
  )
)


In [46]:
# Prepare data
x = df.drop(columns=['PE'])
y = df.drop(columns=['AT', 'V', 'AP', 'RH'])
x_tensor = torch.tensor(x.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)
dataset = TensorDataset(x_tensor, y_tensor)

In [47]:
# Split data into train and validation sets
num_samples = len(dataset)
indices = list(range(num_samples))
split = int(num_samples * 0.8)
np.random.shuffle(indices)
train_indices, val_indices = indices[:split], indices[split:]
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

In [48]:
# Define data loaders
train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=32, sampler=val_sampler)

In [49]:
# Define loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [50]:
# Train the model
def train(dataloader, model, loss_fn, optimizer):
  model.train()
  for batch, (x, y) in enumerate(dataloader):
      x, y = x.to(device), y.to(device)
      optimizer.zero_grad()
      pred = model(x)
      loss = loss_fn(pred, y)
      loss.backward()
      optimizer.step()
      if batch % 100 == 0:
          print(f"Batch {batch}: Loss={loss.item()}")

In [51]:
def test(dataloader, model, loss_fn):
  model.eval()
  total_loss = 0
  with torch.no_grad():
      for x, y in dataloader:
          x, y = x.to(device), y.to(device)
          pred = model(x)
          total_loss += loss_fn(pred, y).item()
  avg_loss = total_loss / len(dataloader)
  print(f"Average test loss: {avg_loss}")

In [54]:
num_epochs = 15
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}")
    train(train_loader, model, loss_fn, optimizer)
    test(val_loader, model, loss_fn)

Epoch 1
Batch 0: Loss=25.775529861450195
Batch 100: Loss=16.492204666137695
Batch 200: Loss=34.06134033203125
Average test loss: 34.727431027094525
Epoch 2
Batch 0: Loss=36.72966003417969
Batch 100: Loss=30.777759552001953
Batch 200: Loss=13.370613098144531
Average test loss: 29.817786820729573
Epoch 3
Batch 0: Loss=39.329715728759766
Batch 100: Loss=23.4320125579834
Batch 200: Loss=31.13399887084961
Average test loss: 28.5551087697347
Epoch 4
Batch 0: Loss=21.97455596923828
Batch 100: Loss=22.577489852905273
Batch 200: Loss=14.564311981201172
Average test loss: 29.41618183453878
Epoch 5
Batch 0: Loss=25.909555435180664
Batch 100: Loss=40.76198196411133
Batch 200: Loss=32.941959381103516
Average test loss: 27.775770902633667
Epoch 6
Batch 0: Loss=29.54888916015625
Batch 100: Loss=24.849246978759766
Batch 200: Loss=39.74842834472656
Average test loss: 27.298509311676025
Epoch 7
Batch 0: Loss=16.56965446472168
Batch 100: Loss=14.243182182312012
Batch 200: Loss=25.838008880615234
Average 